In [8]:
import pandas as pd
import numpy as np

In [2]:
df_sales = pd.read_csv("sales_train_validation.csv")
df_prices = pd.read_csv("sell_prices.csv")
df_calendar = pd.read_csv("calendar.csv")

In [4]:
stores = df_sales.store_id.unique()

In [73]:
def generate_store_df(store_cod):
    df_sales_AUX = df_sales[df_sales["store_id"] == store_cod].copy()
    df_prices_AUX = df_prices[df_prices["store_id"] == store_cod].copy()
    df_calendar_AUX = df_calendar.copy()
    
    df_sales_AUX.drop(["id", "dept_id", "cat_id", "store_id", "state_id"], axis=1, inplace=True)
    df_calendar_AUX = df_calendar_AUX[["wm_yr_wk","date", "d", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]]
    
    df_prices_AUX = df_prices_AUX.pivot(index="wm_yr_wk", columns="item_id", values="sell_price")
    df_prices_AUX.fillna(method="backfill", axis=0, inplace=True)
    
    df_calendar_AUX = df_calendar_AUX.merge(df_prices_AUX, on="wm_yr_wk", how="left")
    df_calendar_AUX.drop("wm_yr_wk", axis=1, inplace=True)
    
    sales_days = ["d_"+str(x) for x in range(1, 1914)]
    df_sales_AUX = pd.melt(df_sales_AUX, id_vars=["item_id"], value_vars=sales_days)
    df_sales_AUX.rename(columns = {'variable':'d'}, inplace=True)
    df_sales_AUX = df_sales_AUX.pivot(index='d', columns='item_id', values='value')
    df_sales_AUX.reset_index(inplace=True)
        
    result = df_sales_AUX.merge(df_calendar_AUX, on="d", how="left")
    result.rename(columns = {'date_y':'date'}, inplace=True)
    result["d"] = result["d"].apply(lambda x : int(x.split("_")[1]))
    result.set_index("d", drop=True, inplace=True)
    result.sort_index(inplace=True)
    result.columns = result.columns.str.replace("_x", "_sells")
    result.columns = result.columns.str.replace("_y", "_price")
    
    return result

In [74]:
datasets = dict()
for s in stores:
    print(s + "\n")
    aux = generate_store_df(s)
    datasets[s] = aux

CA_1

CA_2

CA_3

CA_4

TX_1

TX_2

TX_3

WI_1

WI_2

WI_3



In [76]:
datasets["CA_3"].date

d
1       2011-01-29
2       2011-01-30
3       2011-01-31
4       2011-02-01
5       2011-02-02
           ...    
1909    2016-04-20
1910    2016-04-21
1911    2016-04-22
1912    2016-04-23
1913    2016-04-24
Name: date, Length: 1913, dtype: object